In [150]:
### Import des dépendances
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import pandas as pd
import ast

# Charger le modèle français de spaCy
nlp_fr = spacy.load("fr_core_news_md")

### Test de reconnaissance de NER sur un texte simple
texte_simple = '''Je voudrais aller de Toulouse à Bordeaux.
Comment me rendre à Port-Boulet depuis la gare de Tours ?
Je veux aller voir mon ami Albert à Tours en partant de Bordeaux.'''
doc = nlp_fr(texte_simple)

spacy.displacy.render(doc, style="ent", jupyter=True)

# Le modèle de base reconnait les NER "LOC" (localisation). Le but est d'entrainer un modèle 
# qui puisse distinguer si le NER est un départ ou une destination.

### Chargement du jeu de données à partir d'un fichier CSV pour l'entrainement
# Le CSV contient deux colonnes: "text" pour les phrases et "entities" pour les entités annotées.

# Lecture du dataset depuis un fichier CSV
# Adaptez le séparateur et l'encodage si nécessaire
df = pd.read_csv('../../dataset/raw/initial_training_data.csv', 
                 sep=',',  # Changez ceci si le séparateur est différent
                 encoding='utf-8', 
                 on_bad_lines='skip')  # Ignore les lignes problématiques

# Afficher un aperçu du dataset
df.head()

# Conversion du dataset en format spaCy
from random import shuffle

db = DocBin()  # Le DocBin est un conteneur pour les documents dans spaCy

# Mélange des données pour éviter un apprentissage biaisé
data = list(zip(df['text'], df['entities']))
shuffle(data)

def verify_entity_alignment(text, entities):
    verified_entities = []
    for ent in entities:
        start, end, label = ent['start'], ent['end'], ent['label']
        entity_text = text[start:end]
        # Vérifier si le texte correspond bien à l'entité attendue
        if entity_text == text[start:end]:
            verified_entities.append(ent)
        else:
            print(f"Skipping entity: {entity_text} in text: {text}")
    return verified_entities

for text, entities in data:
    entities = ast.literal_eval(entities)
    verified_entities = verify_entity_alignment(text, entities)
    if verified_entities:
        # Procéder à l'ajout des entités validées
        doc = nlp_fr.make_doc(text)
        ents = []
        for ent in verified_entities:
            start, end, label = ent['start'], ent['end'], ent['label']
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span:
                ents.append(span)
        doc.ents = ents
        db.add(doc)

# Sauvegarder les données d'entrainement au format spaCy
db.to_disk("../../dataset/processed/processed_training_data.spacy")

### Entrainement du modèle

# Cette commande permet de créer un fichier de configuration pour l'entraînement
!python -m spacy init config ./configuration.cfg --lang fr --pipeline ner --optimize efficiency --force

# Lancement de l'entrainement
!python -m spacy train ./configuration.cfg --output ../../models/saved_models --paths.train ../../dataset/processed/processed_training_data.spacy --paths.dev ../../dataset/processed/processed_training_data.spacy

### Chargement et test du modèle

# Chargement du modèle customisé
nlp_itineraire = spacy.load("../../models/saved_models/model-best/")

# Définition des couleurs pour les entités (utilisées lors du rendu avec displacy)
colors = {"DEPARTURE": "#ffe899", "DESTINATION": "#b1ff5e", "ESCALE": "#82b8ff"}
options = {"ents": ["DEPARTURE", "DESTINATION", "ESCALE"], "colors": colors}

# Test du modèle sur le texte simple
doc2 = nlp_itineraire(texte_simple)
spacy.displacy.render(doc2, style="ent", options=options, jupyter=True)

### Test du modèle sur un texte plus compliqué

texte_complique = '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence.
Je compte me rendre à Bordeaux depuis Marseille pour rendre visite à ma soeur Paris.
Je dois regarder les trains Toulouse - Brest pour aller voir mon ami Albert.
Je dois planifier un voyage Nice Toulouse pour les prochaines vacances.
Une réunion de travail m'oblige à faire Paris - Clermont-Ferrand la semaine prochaine.'''

doc3 = nlp_itineraire(texte_complique)
spacy.displacy.render(doc3, style="ent", options=options, jupyter=True)

### Test du modèle sur un texte avec des escales

texte_escale = '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence avec une escale à Nice.
Je compte me rendre à Bordeaux depuis Marseille en m'arrêtant à Toulouse pour rendre visite à ma soeur Paris.
Je dois regarder les trains Toulouse - Brest pour aller voir mon ami Albert en faisant une escale à Tours.
Je dois planifier un voyage Nice Toulouse en passant par Aubervilliers pour les prochaines vacances.
Une réunion de travail m'oblige à faire Paris - Clermont-Ferrand avec un arrêt par Lyon la semaine prochaine.'''

doc4 = nlp_itineraire(texte_escale)
spacy.displacy.render(doc4, style="ent", options=options, jupyter=True)


⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: fr
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
configuration.cfg
You can now add your data and train your pipeline:
python -m spacy train configuration.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory: ..\..\models\saved_models
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       

In [151]:
# Fonction pour afficher des métriques de classification
def evaluate_model(texts, labels, model):
    preds = []
    trues = []
    
    for text, true_labels in zip(texts, labels):
        doc = model(text)
        pred_labels = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
        preds.extend(pred_labels)
        trues.extend(true_labels)
    
    return classification_report(trues, preds, output_dict=True), trues, preds

# Générer des prédictions et afficher un rapport de classification
texts = df['text'].tolist()
entities = [ast.literal_eval(e) for e in df['entities'].tolist()]

metrics, true_labels, pred_labels = evaluate_model(texts, entities, nlp_custom)

# Afficher un rapport de classification
print("Rapport de classification:")
print(classification_report(true_labels, pred_labels))

# Visualiser la matrice de confusion pour les prédictions
ConfusionMatrixDisplay.from_predictions(
    [label[2] for label in true_labels], 
    [label[2] for label in pred_labels], 
    cmap="Blues"
)
plt.title("Matrice de Confusion des Entités")
plt.show()

# Visualisation des métriques avec des graphiques
def plot_metrics(metrics):
    df_metrics = pd.DataFrame(metrics).transpose()
    df_metrics = df_metrics.drop(columns=["support"])

    plt.figure(figsize=(10, 6))
    sns.heatmap(df_metrics, annot=True, fmt=".2f", cmap="YlGnBu")
    plt.title("Rapport de Classification par Catégorie")
    plt.show()

plot_metrics(metrics)

NameError: name 'nlp_custom' is not defined